# Algorytm w oparciu o drzewo rozpinające

#### Funkcja ```ST_algorithm(matrix)``` zwraca kolorowanie i sumę

#### Funkcja ```calculate_sum(matrix)``` zwraca tylko sumę

##### Istotna uwaga: Funkcje przyjmują obiekt ```np.Array``` lub ```np.Matrix```, nie dwuwymiarową listę Pythonową

In [4]:
import math
import numpy as np
import pandas as pd
from collections import deque
import sys
import networkx as nx
import copy
import random
import json
import matplotlib.pyplot as plt

## Funkcje pomocnicze

In [5]:
# Losowa macierz sąsiedztwa
def generate_random_adjacency_matrix(num_nodes, edge_probability):
    matrix = np.zeros((num_nodes, num_nodes), dtype=int)
    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            r = np.random.rand()
            if r <= edge_probability:
                matrix[i][j] = 1
                matrix[j][i] = 1

    return np.asmatrix(matrix)

In [6]:
# Sprawdzenie czy dane kolorowanie jest poprawne
# Przyjmuje kolorwanie jako słownik i macierz sąsiedztwa grafu
def is_correct_vertex_coloring(vertex_coloring, adjacency_matrix):
    # Iterujemy po wierzchołkach
    for vertex, color in vertex_coloring.items():
        # Lista sąsiadów
        neighbors = [i for i, value in enumerate(adjacency_matrix[vertex]) if value == 1]
        for neighbor in neighbors:
            # Jeżeli kolor jest zajęty
            if vertex_coloring[neighbor] == color:
                return False
    return True

In [7]:
# Konstrukcja listy sąsiadów
def construct_neighbor_list(graph, v):
    list = []
    for i in range(len(graph)):
        if graph[v][i] == 1:
            list.append(i)
    return list


# Konstrukcja listy kolorów
def neighb_colors(coloring, neighb):
    neighb = set(neighb)

    colors_list = list(map(lambda key: coloring[key], neighb))

    return colors_list

## Kolejne etapy algorytmu

#### Znalezienie sumy chromatycznej drzewa

In [8]:
# Przyjmuje obiekt grafu z biblioteki networkx
def optimal_tree_coloring(tree):

    # Slowniki {Nr wierzcholka: wartosc}
    # Wierzcholki chcemy numerowac od 1, dlatego i+1
    rcolor_dict = {i+1: None for i in range(tree.number_of_nodes())}
    delta_dict = {i+1: None for i in range(tree.number_of_nodes())}
    ncolor_dict = {i+1: None for i in range(tree.number_of_nodes())}
    minsum_dict = {i+1: None for i in range(tree.number_of_nodes())}

    # Tworzymy słownik rodzicow {Wierzchołek: Lista jego dzieci}
    sons_dict_beta = nx.dfs_successors(tree)
    sons_dict = {}

    # Zwiększamy wartość każdego klucza (zeby pierwszy wierzcholek mial nr 1, nie 0 itd)
    for key in sons_dict_beta:
        new_key = key + 1
        sons_dict[new_key] = sons_dict_beta[key]

    # To samo z lista dzieci
    for key in sons_dict:
        for k in range(len(sons_dict[key])):
            sons_dict[key][k] += 1

    coloradd = [None for _ in range(9999)]

    #print(sons_dict)

    # Chcemy preorder
    preorder_nodes = list(nx.dfs_preorder_nodes(tree))

    # Odwracamy, przez co zawsze odwiedzimy najpierw dzieci danego wierzcholka
    # Przed nim samym
    preorder_nodes.reverse()

    color1 = None
    color2 = None

    for k in preorder_nodes:

        # operujemy na indeksach zwiekszonych o 1
        i = k+1

        # TREE[I].NOSONS = 0 równowazne z tym, ze nie ma ich w slowniku rodzicow
        if i not in sons_dict:
            minsum_dict[i] = 1
            rcolor_dict[i] = 1
            delta_dict[i] = 1
            ncolor_dict[i] = 2

        else:
            # p - iterator po kolejnych dzieciach wierzcholka i
            p = 0
            son = sons_dict[i][p]
            mintotal = 0

            for j in range(1, len(sons_dict[i]) + 2 + 1):  # chcemy ostatnia wartosc wlacznie, wiec dodajemy 1
                coloradd[j] = j

            for j in range(1, len(sons_dict[i]) + 1):

                mintotal = mintotal + minsum_dict[son]
                coloradd[rcolor_dict[son]] += delta_dict[son]

                # SON = TREE[SON].BROTHER
                if not j == len(sons_dict[i]):
                    p += 1
                    son = sons_dict[i][p]

            sum1 = float('inf')
            sum2 = float('inf')

            # Ta deklaracja nie jest konieczna
            color1 = None
            color2 = None

            for j in range(1, len(sons_dict[i]) + 2 + 1):
                value = coloradd[j]
                if value < sum1:
                    color2 = color1
                    sum2 = sum1
                    color1 = j
                    sum1 = value
                else:
                    if value < sum2:
                        color2 = j
                        sum2 = value

            minsum_dict[i] = sum1 + mintotal
            rcolor_dict[i] = color1
            delta_dict[i] = sum2 - sum1
            ncolor_dict[i] = color2
    
    # minsum pierwszego wierzchołka - oczywiscie dodajemy 1
    return minsum_dict[preorder_nodes[-1] + 1]

#### Etap znajdowania kolorowania drzewa

In [9]:
# Bruteforce wykorzystujący backtracking
# Szuka najlepszego kolorowania używając num_colors kolorów
def color_tree(adjacency_matrix, num_colors):

    num_vertices = len(adjacency_matrix)

    colors = [i+1 for i in range(num_colors+1)]

    colored_vertices = {}

    # Sprawdzamy czy kolorowanie wierzchołka na dany kolor jest poprawne
    def is_valid(vertex, color):
        for neighbor in range(num_vertices):
            if adjacency_matrix[vertex][neighbor] == 1 and neighbor in colored_vertices and colored_vertices[neighbor] == color:
                return False
        return True


    def backtrack(vertex):
        if vertex == num_vertices:
            return True

        for color in colors:
            # Jeżeli można pokolorwać, to kolorujemy
            if is_valid(vertex, color):
                colored_vertices[vertex] = color
                if backtrack(vertex + 1):
                    return True
                del colored_vertices[vertex]

        return False

    # Zaczynamy od wierzchołka 0
    if backtrack(0):
        return colored_vertices
    else:
        return None

# Wykorzystanie wcześniejszej funkcji by zbadać najlepsze kolorowanie
# używając ustalonej maksymalnej liczby kolorów
# num_colorings - liczba iteracji (prób kolorowań)
# tolerance - tolerancja w oparciu o sumę chromatyczną
def color_tree_multiple(adjacency_matrix, num_colors, num_colorings, tolerance):

    best_sum = float('inf')
    best_coloring = {}

    for _ in range(num_colorings):

        # Kolorujemy w pewien sposób drzewo wykorzystując num_colors kolorów
        coloring = color_tree(adjacency_matrix, num_colors)
        temp_sum = sum(coloring.values())

        # Jeżeli jest wystarczająco dobre
        if temp_sum <= tolerance:
            return coloring

        # Jeżeli jest lepsze niż jakiekolwiek dotychczas
        if temp_sum < best_sum:
            best_coloring = coloring
            best_sum = temp_sum
    return best_coloring

#### Etap dodawania krawędzi do drzewa rozpinającego

In [10]:
# tree - drzewo rozpinające (m. sąsiedztwa)
# graph - graf docelowy (m. sąsiedztwa)
# tree_coloring - słownik kolorowania tree
def add_edges(tree, graph, tree_coloring):

    coloring = tree_coloring.copy()

    # [graph - tree] to wymagane krawędzie - macierz edges
    edges = graph - tree

    # Zamiana na listy
    tree = tree.tolist()
    graph = graph.tolist()
    edges = edges.tolist()
    constr_graph = tree
    # Bierzemy krawędź (a,b) (tam gdzie edges == 1)

    for i in range(len(graph)):
        for j in range(len(graph)):
            if edges[i][j] == 1:

                # Zmieniamy, jeżeli psulibysmy kolorowanie
                if coloring[i] == coloring[j]:
                    
                    # Listy sąsiadów
                    neighb_i = construct_neighbor_list(constr_graph, i)
                    neighb_j = construct_neighbor_list(constr_graph, j)

                    # Lista kolorów sąsiadów
                    neighb_col_i = neighb_colors(coloring, neighb_i)
                    neighb_col_j = neighb_colors(coloring, neighb_j)


                    # Najmniejsze możliwe kolorowania
                    min_possible_i = max(neighb_col_i) + 1

                    for p in range(max(neighb_col_i) + 1):
                        if (p + 1) not in neighb_col_i:
                            min_possible_i = p + 1
                            break

                    min_possible_j = max(neighb_col_j) + 1

                    for p in range(max(neighb_col_j)+1):
                        if p + 1 not in neighb_col_j:
                            min_possible_j = p + 1
                            break
                            
                    # Docelowo zawsze zmieniamy i
                    # Jeżeli potrzeba to zamieniamy wartości
                    if min_possible_j < min_possible_i:
                        i, j = j, i
                        min_possible_i, min_possible_j = min_possible_j, min_possible_i
                        neighb_col_i, neighb_col_j = neighb_col_j, neighb_col_i

                    if coloring[i] == min_possible_i:
                        while True:
                            min_possible_i += 1
                            if min_possible_i not in neighb_col_i:
                                coloring[i] = min_possible_i
                                break
                            if min_possible_i not in neighb_col_j:
                                coloring[j] = min_possible_i
                                break
                    else:
                        # Update coloringu
                        coloring[i] = min_possible_i

            # Aktualizacja macierzy
            edges[i][j], edges[j][i] = 0, 0
            constr_graph[i][j], constr_graph[j][i] = 1, 1
    return constr_graph, coloring

#### Etap poprawiania liści

In [11]:
def correct_leaves(graph, coloring):

    index = 0
    coloring_res = coloring.copy()
    for i in range(len(graph)):

        # Znalezienie stopnia 1
        if sum(graph[i]) == 1:

            # Sprawdzenie kto jest jego jedynym sąsiadem
            for j in range(len(graph[i])):
                if graph[i][j] == 1:
                    index = j
                    break

            # Docelowo chcemy zamienic liscie na 1, ale
            # gdy sasiaduje juz z 1 to musimy na 2
            if coloring_res[index] == 1:
                coloring_res[i] = 2
            else:
                coloring_res[i] = 1

    return coloring_res

In [18]:
def ST_algorithm(matrix):
    
    graph = nx.from_numpy_array(matrix)
    tree = nx.minimum_spanning_tree(graph)
    
    # Suma chromatyczna drzewa
    coloring = optimal_tree_coloring(tree)
    
    tolerance = 1.1 * coloring
    
    #Najmniejsza znaleziona suma kolorów
    min_suma = float('inf')
    best_coloring = {}

    # k to dopuszczalna liczba iteracji bruteforce
    k = 4000

    # i to numer koloru
    # Szukanie 
    for i in range(2, 6):
        bruteforce = color_tree_multiple(nx.to_numpy_array(tree), i, k, tolerance)
        suma = sum(bruteforce.values())
        if suma < min_suma:
            best_coloring = bruteforce
            min_suma = suma

    tree_adj = nx.to_numpy_array(tree)

    # Etap dodawania krawędzi
    end_graph, end_coloring = add_edges(tree_adj, matrix, best_coloring)

    # Poprawianie liści
    end_coloring = correct_leaves(end_graph, end_coloring)

           # Kolorowanie, wyznaczona suma kolorów
    return end_coloring, sum(end_coloring.values())

In [19]:
def calculate_sum(matrix):

    coloring, sum = ST_algorithm(matrix)

    return sum

In [45]:
matrix = generate_random_adjacency_matrix(15, 0.8)
coloring, suma = ST_algorithm(matrix)

In [46]:
is_correct_vertex_coloring(coloring, matrix.tolist())

True

In [49]:
matrix

matrix([[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1],
        [1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0],
        [1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1],
        [1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1],
        [1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1],
        [1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1],
        [1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1],
        [1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1],
        [1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1],
        [0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1],
        [1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1],
        [1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1],
        [1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0]])

In [47]:
coloring

{0: 1,
 1: 2,
 2: 3,
 3: 2,
 4: 4,
 5: 5,
 6: 2,
 7: 6,
 8: 3,
 9: 7,
 10: 1,
 11: 8,
 12: 9,
 13: 10,
 14: 11}

In [48]:
suma

74